In [48]:
from requests import Session, Request
from numba import njit
import pandas as pd
import numpy as np
from itertools import cycle
from bs4 import BeautifulSoup
import json
from pprint import pprint as pp
import re
%load_ext line_profiler
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [79]:
class UserAgent:

    user_agents_ = [
        "Mozilla/5.0 (Linux; Android 8.0.0; SM-G960F Build/R16NW) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.84 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 7.0; SM-G892A Build/NRD90M; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/60.0.3112.107 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 7.0; SM-G930VC Build/NRD90M; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/58.0.3029.83 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 6.0.1; SM-G935S Build/MMB29K; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/55.0.2883.91 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 6.0.1; SM-G920V Build/MMB29K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.98 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 5.1.1; SM-G928X Build/LMY47X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 6.0.1; Nexus 6P Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 7.1.1; G8231 Build/41.2.A.0.219; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/59.0.3071.125 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 6.0.1; E6653 Build/32.2.A.0.253) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.98 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 6.0; HTC One X10 Build/MRA58K; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/61.0.3163.98 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 6.0; HTC One M9 Build/MRA58K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.98 Mobile Safari/537.3",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 12_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 12_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/69.0.3497.105 Mobile/15E148 Safari/605.1",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 12_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) FxiOS/13.2b11866 Mobile/16A366 Safari/605.1.15",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.34 (KHTML, like Gecko) Version/11.0 Mobile/15A5341f Safari/604.1",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A5370a Safari/604.1",
        "Mozilla/5.0 (iPhone9,3; U; CPU iPhone OS 10_0_1 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) Version/10.0 Mobile/14A403 Safari/602.1",
        "Mozilla/5.0 (iPhone9,4; U; CPU iPhone OS 10_0_1 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) Version/10.0 Mobile/14A403 Safari/602.1",
        "Mozilla/5.0 (Apple-iPhone7C2/1202.466; U; CPU like Mac OS X; en) AppleWebKit/420+ (KHTML, like Gecko) Version/3.0 Mobile/1A543 Safari/419.3",
        "Mozilla/5.0 (Windows Phone 10.0; Android 6.0.1; Microsoft; RM-1152) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Mobile Safari/537.36 Edge/15.15254",
        "Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; RM-1127_16056) AppleWebKit/537.36(KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10536",
        "Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; Lumia 950) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2486.0 Mobile Safari/537.36 Edge/13.1058",
        "Mozilla/5.0 (Linux; Android 7.0; Pixel C Build/NRD90M; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/52.0.2743.98 Safari/537.36",
        "Mozilla/5.0 (Linux; Android 6.0.1; SGP771 Build/32.2.A.0.253; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/52.0.2743.98 Safari/537.36",
        "Mozilla/5.0 (Linux; Android 6.0.1; SHIELD Tablet K1 Build/MRA58K; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/55.0.2883.91 Safari/537.36",
        "Mozilla/5.0 (Linux; Android 7.0; SM-T827R4 Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.116 Safari/537.36",
        "Mozilla/5.0 (Linux; Android 5.0.2; SAMSUNG SM-T550 Build/LRX22G) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/3.3 Chrome/38.0.2125.102 Safari/537.36",
        "Mozilla/5.0 (Linux; Android 4.4.3; KFTHWI Build/KTU84M) AppleWebKit/537.36 (KHTML, like Gecko) Silk/47.1.79 like Chrome/47.0.2526.80 Safari/537.36",
        "Mozilla/5.0 (Linux; Android 5.0.2; LG-V410/V41020c Build/LRX22G) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/34.0.1847.118 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246",
        "Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36",
        "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1"
        "Mozilla/5.0 (CrKey armv7l 1.5.16041) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1650.0 Safari/537.36",
        "Roku4640X/DVP-7.70 (297.70E04154A)",
        "Mozilla/5.0 (Linux; U; Android 4.2.2; he-il; NEO-X5-116A Build/JDQ39) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 Safari/534.30",
        "Mozilla/5.0 (Linux; Android 5.1; AFTS Build/LMY47O) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/41.99900.2250.0242 Safari/537.36",
        "Dalvik/2.1.0 (Linux; U; Android 6.0.1; Nexus Player Build/MMB29T)",
        "AppleTV6,2/11.1",
        "AppleTV5,3/9.1.1",
        "Mozilla/5.0 (Nintendo WiiU) AppleWebKit/536.30 (KHTML, like Gecko) NX/3.0.4.2.12 NintendoBrowser/4.3.1.11264.US",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; XBOX_ONE_ED) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393",
        "Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Xbox; Xbox One) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2486.0 Mobile Safari/537.36 Edge/13.10586",
        "Mozilla/5.0 (PlayStation 4 3.11) AppleWebKit/537.73 (KHTML, like Gecko)",
        "Mozilla/5.0 (PlayStation Vita 3.61) AppleWebKit/537.73 (KHTML, like Gecko) Silk/3.2",
        "Mozilla/5.0 (Nintendo 3DS; U; ; en) Version/1.7412.EU",
        "Mozilla/5.0 (X11; U; Linux armv7l like Android; en-us) AppleWebKit/531.2+ (KHTML, like Gecko) Version/5.0 Safari/533.2+ Kindle/3.0+",
        "Mozilla/5.0 (Linux; U; en-US) AppleWebKit/528.5+ (KHTML, like Gecko, Safari/528.5+) Version/4.0 Kindle/3.0 (screen 600x800; rotate)"
    ]


    def __init__(self, user_agents_=user_agents_):
        self.user_agents_ = iter(user_agents_)
    
    def get_new_ua(self):
        return next(self.user_agents_)

In [80]:
ua = UserAgent()
for each in range(5):
    print(ua.get_new_ua())

Mozilla/5.0 (Linux; Android 8.0.0; SM-G960F Build/R16NW) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.84 Mobile Safari/537.36
Mozilla/5.0 (Linux; Android 7.0; SM-G892A Build/NRD90M; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/60.0.3112.107 Mobile Safari/537.36
Mozilla/5.0 (Linux; Android 7.0; SM-G930VC Build/NRD90M; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/58.0.3029.83 Mobile Safari/537.36
Mozilla/5.0 (Linux; Android 6.0.1; SM-G935S Build/MMB29K; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/55.0.2883.91 Mobile Safari/537.36
Mozilla/5.0 (Linux; Android 6.0.1; SM-G920V Build/MMB29K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.98 Mobile Safari/537.36


In [45]:
class ScrapeData:
    def __init__(self):
        self.set_constants()
        self.establish_session()
        self.data_ = self.scrape_countries()
    
    def set_constants(self):
        self.base_url = 'https://www.wine-searcher.com'
        #'''
        self.headers_ = {"User-Agent": "Mozilla/5.0 (Nintendo WiiU) AppleWebKit/536.30 (KHTML, like Gecko) NX/3.0.4.2.12 NintendoBrowser/4.3.1.11264.US",
                   "path" : "/regions",
                    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                   "Accept-Language": "en-US,en;q=0.5",
                   "Accept-Encoding": "gzip, deflate",
                   "DNT": "1",
                   "sec-fetch-dest" : "document",
                   "sec-fetch-mode" : "navigate",
                   "Connection": "close",
                   "Upgrade-Insecure-Requests": "1",
                    "Cache-Control" : 'no-cache'}
        '''
        self.headers_ = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36",
                        'Cache-Control' : 'no-cache'}
        '''
    
    def establish_session(self):
        self.session_ = Session()
        self.ua = UserAgent()
    
    def scrape_countries(self):
        while True:
            countries_response_ = self.session_.get(f'{self.base_url}/regions', headers=self.headers_)
            if countries_response_.status_code == 403:
                self.change_ua()
                #return f"403 in {country_}/region", "Error"
            else:
                break
        
        if countries_response_.status_code == 403:
            return "403 in country"
        countries_response_bs_ = BeautifulSoup(countries_response_.content, 'html.parser')
        countries_eps_ = countries_response_bs_.find(string='Top countries:').find_parent("div").next_sibling.next_sibling
        countries_eps_ = [[ctry_.text, ctry_.a["href"]] for ctry_ in countries_eps_ if not str(ctry_)=="\n"]
        list_countries_ = []
        for country_, country_endpoint_ in countries_eps_[:1]:
            each_country_ = {}
            response_, label_ = self.scrape_regions(country_, country_endpoint_)
            each_country_[country_] = [{'country_endpoint' : country_endpoint_},
                                       {label_ : response_}]
            list_countries_.append(each_country_)
        return list_countries_
        
    
    def scrape_regions(self, country_, country_endpoint_):
        while True:
            regions_response_ = self.session_.get(f'{self.base_url}/{country_endpoint_}', headers=self.headers_)
            if regions_response_.status_code == 403:
                self.change_ua()
                #return f"403 in {country_}/region", "Error"
            else:
                break
        regions_response_bs_ = BeautifulSoup(regions_response_.content, 'html.parser')
        regions_eps_ = regions_response_bs_.find(string=f'Sub-regions of {country_}')
        if not regions_eps_:
            wines_, label_ = self.scrape_wines(country_, country_endpoint_, loc_content=regions_response_bs_)
            return wines_, label_
        regions_eps_ = regions_eps_.find_parent("div").next_sibling.next_sibling
        regions_eps_ = [[reg_.text, reg_.a["href"]] for reg_ in regions_eps_.children if not str(reg_)=='\n']
        list_regions_ = []
        for region_, region_endpoint_ in regions_eps_:
            each_region_ = {}
            response_, label_ = self.scrape_subregions(region_, region_endpoint_)
            each_region_[region_] = [{'region_endpoint' : region_endpoint_},
                                       {label_ : response_}]
            list_regions_.append(each_region_)
        return list_regions_, "regions"
    
    def scrape_subregions(self, region_, region_endpoint_):
        while True:
            subregions_response_ = self.session_.get(f'{self.base_url}/{region_endpoint_}', headers=self.headers_)
            if subregions_response_.status_code == 403:
                self.change_ua()
                #return f"403 in {region_}/subregion", "Error"
            else:
                break
        subregions_response_bs_ = BeautifulSoup(subregions_response_.content, 'html.parser')
        subregions_eps_ = subregions_response_bs_.find(string=f'Sub-regions of {region_}')
        if not subregions_eps_:
            wines_, label_ = self.scrape_wines(region_, region_endpoint_, loc_content=subregions_response_bs_)
            return wines_, label_
        subregions_eps_ = subregions_eps_.find_parent("div").next_sibling.next_sibling
        subregions_eps_ = [[subreg_.text, subreg_.a["href"]] for subreg_ in subregions_eps_.children if not str(subreg_)=='\n']
        list_subregions_ = []
        for subregion_, subregion_endpoint_ in subregions_eps_:
            each_subregion_ = {}
            response_, label_ = self.scrape_appelations(subregion_, subregion_endpoint_)
            each_subregion_[subregion_] = [{'subregion_endpoint' : subregion_endpoint_},
                                       {label_ : response_}]
            list_subregions_.append(each_subregion_)
        return list_subregions_, "sub-regions"
    
    def scrape_appelations(self, subregion_, subregion_endpoint_):
        while True:
            appelations_response_ = self.session_.get(f'{self.base_url}/{subregion_endpoint_}', headers=self.headers_)
            if appelations_response_.status_code == 403:
                self.change_ua()
                #return f"403 in {subregion_}/appelation", "Error"
            else:
                break
        appelations_response_bs_ = BeautifulSoup(appelations_response_.content, 'html.parser')
        appelations_eps_ = appelations_response_bs_.find(string=f'Sub-regions of {subregion_}')
        if not subregions_eps_:
            wines_, label_ = self.scrape_wines(subregion_, subregion_endpoint_, loc_content=appelations_response_bs_)
            return wines_, label_
        appelations_eps_ = appelations_eps_.find_parent("div").next_sibling.next_sibling
        appelations_eps_ = [[app_.text, app_.a["href"]] for app_ in appelations_eps_.children if not str(app_)=='\n']
        list_appelations_ = []
        for appelation_, appelation_endpoint_ in appelations_eps_:
            each_appelation_ = {}
            response_, label_ = self.scrape_vineyards(appelation_, appelation_endpoint_)
            each_appelation_[appelation_] = [{'appelation_endpoint' : appelation_endpoint_},
                                       {label_ : response_}]
            list_appelations_.append(each_appelation_)
        return list_appelations_, "appelations"
    
    def scrape_vineyards(self, appelation_, appelation_endpoint_):
        while True:
            vineyards_response_ = self.session_.get(f'{self.base_url}/{appelation_endpoint_}', headers=self.headers_)
            if vineyards_response_.status_code == 403:
                self.change_ua()
                #return f"403 in {appelation_}/vineyard", "Error"
            else:
                break
        vineyards_response_bs_ = BeautifulSoup(vineyards_response_.content, 'html.parser')
        vineyards_eps_ = vineyards_response_bs_.find(string=f'Sub-regions of {appelation_}')
        if not vineyards_eps_:
            wines_, label_ = self.scrape_wines(appelation_, appelation_endpoint_)
            return wines_, label_
        vineyards_eps_ = vineyards_eps_.find_parent("div").next_sibling.next_sibling
        vineyards_eps_ = [[vins_.text, vins_.a["href"]] for vins_ in vineyards_eps_.children if not str(vins_)=='\n']
        list_vineyards_ = []
        for vineyard_, vineyard_endpoint_ in vineyards_eps_:
            each_vineyard_ = {}
            response_, label_ = self.scrape_wines(vineyard_, vineyard_endpoint_)
            each_vineyard_[vineyard_] = [{'vineyard_endpoint' : vineyard_endpoint_},
                                       {label_ : response_}]
            list_vineyards_.append(each_vineyard_)
        return list_vineyards_, "vineyards"
    
    def scrape_wines(self, loc_, loc_endpoint_, loc_content=None, from_vineyard_=False):
        if from_vineyard_:
            wines_response_ = self.session_.get(f'{self.base_url}/{loc_endpoint_}', headers=self.headers_)
            if wines_response_.status_code == 403:
                return f"403 in {loc_}/wine", "Error"
            wines_response_bs_ = BeautifulSoup(wines_response_.content, 'html.parser')
        else:
            wines_response_bs_ = loc_content
        wines_eps_ = wines_response_bs_.find(string=re.compile(f'Most Popular {loc_} Wine'))
        if not wines_eps_:
            wines_ = [{'No wines' : []}]
            return wines_, "No wines"
        wines_eps_ = wines_eps_.find_parent("h2").find_next_sibling("table").tbody
        wines_eps_ = [row_ for row_ in wines_eps_.children if not str(row_)=='\n']
        list_wines_ = []
        for wine_ in wines_eps_[1:]:
            wine_row_ = wine_.find_all("td")
            wine_info_ = {}
            try:
                wine_info_["Name"] = wine_row_[0].a.text
                wine_info_["Link"] = wine_row_[0].a["href"]
                wine_info_["Grape"] = wine_row_[1].a.text
                wine_info_["Popularity"] = wine_row_[2].text
                wine_info_["Score"] = wine_row_[3].text
                wine_info_["Price"] = wine_row_[4].find_all("span")[1].b.text
            except:
                print(list_wines_)
                print(wine_)
                print(wine_row_)
                print(wine_info_)
                pass
            list_wines_.append(wine_info_)
        return list_wines_, "wines"
    
    def change_ua(self):
        new_ua_ = self.ua.get_new_ua()
        print(f"Changing User-Agent to {new_ua_}")
        self.headers_["User-Agent"] = new_ua_

In [46]:
exp_ = ScrapeData()

[{'Name': "Valle de la Puerta 'La Puerta' Gran Reserva, Famatina, Argentina", 'Link': '/find/valley+de+la+puerta+grand+rsrv+famatina+argentina', 'Grape': 'Rare Red Blend', 'Popularity': '20,280th', 'Score': '89', 'Price': '35'}, {'Name': 'Raza Reserva Malbec, Famatina, Argentina', 'Link': '/find/raza+rsrv+malbec+famatina+argentina', 'Grape': 'Malbec', 'Popularity': '25,464th', 'Score': '', 'Price': '12'}, {'Name': "Valle de la Puerta 'La Puerta' Malbec, Famatina, Argentina", 'Link': '/find/valley+de+la+puerta+malbec+famatina+argentina', 'Grape': 'Malbec', 'Popularity': '30,317th', 'Score': '88', 'Price': '9'}, {'Name': 'Raza Argentina Malbec, Famatina, Argentina', 'Link': '/find/raza+malbec+famatina+argentina', 'Grape': 'Malbec', 'Popularity': '47,328th', 'Score': '87', 'Price': '9'}, {'Name': "Valle de la Puerta 'La Puerta Alta' Malbec - Bonarda, Famatina, Argentina", 'Link': '/find/valley+de+la+puerta+alta+malbec+bonarda+famatina+argentina', 'Grape': 'Bonarda  -  Malbec', 'Popularity

In [47]:
pp(exp_.data_)

[{'Argentina': [{'country_endpoint': '/regions-argentina'},
                {'regions': [{'Cafayate - Calchaqui Valley': [{'region_endpoint': '/regions-cafayate+-+calchaqui+valley'},
                                                              {'wines': [{'Grape': 'Malbec',
                                                                          'Link': '/find/st+pedro+de+yacochuya+cafayate+calchaqui+valley+argentina',
                                                                          'Name': 'San '
                                                                                  'Pedro '
                                                                                  'de '
                                                                                  'Yacochuya '
                                                                                  "'Yacochuya', "
                                                                                  'Cafayate, '
                   

                                           {'Error': '403 in San '
                                                     'Juan/subregion'}]}]}]}]


In [ ]:
sd = ScrapeData()

In [9]:
#data_ = json.dumps(sd.data_, indent=2)
pp(sd.data_)


NameError: name 'sd' is not defined

In [ ]:
df = pd.DataFrame({"name": ['Alfred', 'Batman', 'Catwoman'],
                   "toy": [np.nan, 2, 3],
                   "born": [np.nan, 10,
                            np.nan]})
df["born"].fillna(pd.Series([1, 2]), inplace=True)
print(df)